## OR4 Coadd Metric Viewer

Displays metrics associated with deepCoadds as a series of tables. Metrics that deviate from the column mean by more than 3 standard deviations are highlighted in red.

Bear in mind highlighting assumes a gaussian-ish distribution, meaning that some outlier values may not be highlighted (and vice versa).

Columns named after a band show the number of exposures of that band that have gone into making the coadd (so, "200" under "g" for tract row "500" means that that 200 g-band exposures have gone into creating all the coadds that cover tract 500). 

In [1]:
day_obs = '20240625'
repo = 'embargo_or4'
collection = 'LSSTComCamSim/runs/nightlyValidation/20240625/w_2024_25/DM-44966'

In [2]:
import lsst.daf.butler as dafButler

import numpy as np
import pandas as pd
from scipy.stats import zscore

from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-child {overflow: scroll}</style>"))

In [3]:
def metricBundletToDataFrame(bundleName, refs):
    data = []
    for ref in refs:
        metrics = butler.get(ref)
        row = {'Tract':ref.dataId['tract']}
        for metric in metrics[bundleName]:
            key = '{} {}'.format(metric.metric_name.metric, metric.quantity.unit.to_string())
            value = metric.quantity.value
            row[key] = value
        data.append(row)
    
    df = pd.DataFrame(data)
    sigma = pd.DataFrame(abs(zscore(df, axis=0, nan_policy='omit')), columns=df.columns)
    sigma['Tract'] = df['Tract']
    return (df, sigma)

def displayMaskedTable(tractTable, tractMask, metricTable, metricMask):
    tractMetricTable = pd.merge(tractTable, metricTable, on="Tract")
    tractMetricMask = pd.merge(tractMask, metricMask, on="Tract")
    tractMetricMask['Tract'] = False
    
    style_df = tractMetricMask.map(lambda x: "background-color: red" if x>3 else 
                                   ("background-color: orange" if x>2 else ""))
    with pd.option_context('display.max_rows', 100, 'display.max_columns', 50):
        display(tractMetricTable.style.apply(lambda x: style_df, axis=None))

In [4]:
butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry

In [5]:
visits = list(registry.queryDimensionRecords('visit', where='day_obs in ({})'.format(day_obs)))
visitIds = [visit.id for visit in visits]
minId, maxId = np.min(visitIds), np.max(visitIds)

coaddRefs = list(registry.queryDatasets('deepCoadd_calexp', where='visit in ({}..{})'.format(minId, maxId)))
data = []
for coaddRef in coaddRefs:
    data.append({'Tract':coaddRef.dataId['tract'], 'Band':coaddRef.dataId['band']})
tractTable = pd.DataFrame(data)
tractTable = tractTable.groupby(tractTable.columns.tolist(),as_index=False).size()
tractTable = tractTable.pivot(index='Tract', columns='Band', values='size')
tractMask = tractTable.copy()
for col in tractMask.columns:
    if col != 'Tract':
        tractMask[col].values[:] = 0
tractMask = tractMask.astype(bool)

## Object Table Core Metrics:

In [6]:
refs = list(registry.queryDatasets('objectTableCore_metrics'))
refs

[DatasetRef(DatasetType('objectTableCore_metrics', {skymap, tract}, MetricMeasurementBundle), {skymap: 'ops_rehersal_prep_2k_v1', tract: 9880}, run='LSSTComCamSim/runs/nightlyValidation/test1a/d_2024_06_21/DM-44966/20240625T202549Z', id=41f9eb7d-5dfe-4f8a-8c7f-f0b6fc71a9ac),
 DatasetRef(DatasetType('objectTableCore_metrics', {skymap, tract}, MetricMeasurementBundle), {skymap: 'ops_rehersal_prep_2k_v1', tract: 9638}, run='LSSTComCamSim/runs/nightlyValidation/test1a/d_2024_06_21/DM-44966/20240625T202549Z', id=b04a2326-963f-4980-a43b-fe60ab9e9ca2),
 DatasetRef(DatasetType('objectTableCore_metrics', {skymap, tract}, MetricMeasurementBundle), {skymap: 'ops_rehersal_prep_2k_v1', tract: 2876}, run='LSSTComCamSim/runs/nightlyValidation/20240625/w_2024_25/DM-44966/20240626T155223Z', id=90ebce90-0d8e-485a-9bdc-56f7f545145f),
 DatasetRef(DatasetType('objectTableCore_metrics', {skymap, tract}, MetricMeasurementBundle), {skymap: 'ops_rehersal_prep_2k_v1', tract: 6568}, run='LSSTComCamSim/runs/night

### Blend Metrics

In [7]:
#BlendLogL: Log likelihood of the entire blend in scarlet_lite.
bundleName = 'blendMetrics'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,numBlends,meanBlendIterations,meanBlendLogL
0,2661,146.000000,117.000000,105.000000,1998.000000,21.435936,-202962.062500
1,2662,726.000000,450.000000,751.000000,7040.000000,22.801562,-464628.156250
2,2703,nan,6.000000,12.000000,63.000000,17.190476,-6650.877441
3,2704,nan,nan,23.000000,50.000000,16.720000,-2471.205078
4,2876,562.000000,662.000000,557.000000,5334.000000,16.990626,-8838.389648
5,2877,2424.000000,2384.000000,2493.000000,18794.000000,17.097105,-10273.656250
6,3345,272.000000,276.000000,330.000000,6452.000000,21.465902,-161080.593750
7,3346,983.000000,1104.000000,1063.000000,14912.000000,21.675295,-79525.296875
8,3384,9506.000000,11103.000000,8263.000000,40524.000000,19.858701,-38701.160156
9,3385,257.000000,454.000000,302.000000,2259.000000,20.136786,-9591.259766


### E1Diff Metrics

In [8]:
# Ellipticity Residuals (e1 - e1_psf) per tract per band
# highSNStars are SNR > 2700, lowSNStars are SNR > 500
# _count is number of stars in each SNR bin 
bundleName = 'e1Diff'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct
0,2661,146.000000,117.000000,105.000000,0.001670,0.002904,35.000000,0.001348,0.004498,783.000000,-0.001903,0.002209,16.000000,-0.001293,0.003520,535.000000,-0.000838,0.001922,45.000000,-0.000398,0.003743,930.000000
1,2662,726.000000,450.000000,751.000000,0.000891,0.002761,337.000000,0.001118,0.003939,3715.000000,0.000296,0.001688,479.000000,0.000067,0.003083,4097.000000,-0.000279,0.001901,314.000000,-0.000298,0.003413,3230.000000
2,2703,nan,6.000000,12.000000,nan,nan,0.000000,nan,nan,0.000000,nan,nan,0.000000,0.002644,0.001870,4.000000,nan,nan,0.000000,0.010098,0.013065,3.000000
3,2704,nan,nan,23.000000,nan,nan,0.000000,nan,nan,0.000000,-0.003718,0.000000,1.000000,0.001124,0.004401,4.000000,nan,nan,0.000000,nan,nan,0.000000
4,2876,562.000000,662.000000,557.000000,-0.000069,0.001538,23.000000,-0.000055,0.003006,177.000000,0.000367,0.001355,30.000000,0.000335,0.002995,231.000000,-0.000352,0.002313,43.000000,-0.000269,0.004178,313.000000
5,2877,2424.000000,2384.000000,2493.000000,-0.000197,0.001593,132.000000,-0.000079,0.002783,634.000000,0.000345,0.001663,178.000000,0.000056,0.002733,863.000000,0.000286,0.001995,181.000000,0.000142,0.003064,955.000000
6,3345,272.000000,276.000000,330.000000,0.000136,0.002112,218.000000,0.000263,0.003221,2512.000000,-0.000024,0.001597,350.000000,-0.000043,0.002615,2949.000000,-0.000204,0.001491,236.000000,-0.000227,0.002885,2777.000000
7,3346,983.000000,1104.000000,1063.000000,-0.000093,0.001567,793.000000,0.000042,0.002770,6374.000000,0.000144,0.001453,990.000000,0.000022,0.002699,7484.000000,-0.000025,0.001655,976.000000,0.000001,0.003063,7984.000000
8,3384,9506.000000,11103.000000,8263.000000,0.000093,0.001218,2082.000000,0.000120,0.002259,9161.000000,0.000060,0.001314,2222.000000,0.000031,0.002401,10849.000000,-0.000089,0.001198,2815.000000,-0.000065,0.002309,13103.000000
9,3385,257.000000,454.000000,302.000000,-0.000161,0.002044,31.000000,0.000020,0.002893,367.000000,0.000687,0.001830,38.000000,0.000225,0.003557,563.000000,-0.000202,0.001617,51.000000,0.000156,0.003385,695.000000


### E2 Diff Metrics

In [9]:
bundleName = 'e2Diff'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct
0,2661,146.000000,117.000000,105.000000,0.000142,0.001529,35.000000,0.000285,0.003600,783.000000,-0.000827,0.001089,16.000000,-0.001161,0.003248,535.000000,-0.000414,0.002880,45.000000,-0.000231,0.004279,930.000000
1,2662,726.000000,450.000000,751.000000,0.000044,0.001462,337.000000,0.000112,0.002880,3715.000000,-0.000358,0.001565,479.000000,-0.000277,0.002916,4097.000000,-0.000038,0.001842,314.000000,-0.000242,0.003507,3230.000000
2,2703,nan,6.000000,12.000000,nan,nan,0.000000,nan,nan,0.000000,nan,nan,0.000000,0.001642,0.007396,4.000000,nan,nan,0.000000,-0.001738,0.000809,3.000000
3,2704,nan,nan,23.000000,nan,nan,0.000000,nan,nan,0.000000,-0.003200,0.000000,1.000000,0.001612,0.006113,4.000000,nan,nan,0.000000,nan,nan,0.000000
4,2876,562.000000,662.000000,557.000000,-0.000674,0.001489,23.000000,0.000189,0.002949,177.000000,0.000108,0.002049,30.000000,0.000029,0.002794,231.000000,-0.000708,0.002197,43.000000,-0.000034,0.003551,313.000000
5,2877,2424.000000,2384.000000,2493.000000,0.000599,0.001657,132.000000,0.000060,0.002728,634.000000,0.000014,0.002155,178.000000,0.000060,0.002792,863.000000,0.000050,0.001920,181.000000,0.000122,0.002982,955.000000
6,3345,272.000000,276.000000,330.000000,-0.000292,0.001875,218.000000,-0.000428,0.003317,2512.000000,-0.000359,0.001438,350.000000,-0.000376,0.002561,2949.000000,0.000490,0.001434,236.000000,0.000007,0.002923,2777.000000
7,3346,983.000000,1104.000000,1063.000000,-0.000304,0.001610,793.000000,-0.000240,0.002829,6374.000000,-0.000125,0.001441,990.000000,-0.000172,0.002636,7484.000000,0.000329,0.001838,976.000000,0.000202,0.003171,7984.000000
8,3384,9506.000000,11103.000000,8263.000000,0.000078,0.001137,2082.000000,0.000118,0.002106,9161.000000,-0.000028,0.001336,2222.000000,-0.000035,0.002317,10849.000000,-0.000029,0.001229,2815.000000,-0.000026,0.002324,13103.000000
9,3385,257.000000,454.000000,302.000000,0.000195,0.001487,31.000000,-0.000293,0.002778,367.000000,0.000609,0.001998,38.000000,-0.000284,0.003914,563.000000,-0.000225,0.002169,51.000000,-0.000176,0.003427,695.000000


### Isolated Deblender Metrics

In [10]:
bundleName = 'isolatedDeblenderMetrics'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,numIsolated,meanIsolatedIterations,meanIsolatedLogL
0,2661,146.000000,117.000000,105.000000,4486.000000,17.182791,-10030.127930
1,2662,726.000000,450.000000,751.000000,12951.000000,17.160991,-3143.437988
2,2703,nan,6.000000,12.000000,380.000000,16.450000,-300.461487
3,2704,nan,nan,23.000000,411.000000,16.335766,-252.150620
4,2876,562.000000,662.000000,557.000000,23715.000000,16.201223,-424.545135
5,2877,2424.000000,2384.000000,2493.000000,64652.000000,16.186661,-441.756165
6,3345,272.000000,276.000000,330.000000,15908.000000,16.875911,-1666.786255
7,3346,983.000000,1104.000000,1063.000000,32149.000000,16.795235,-490.236053
8,3384,9506.000000,11103.000000,8263.000000,94790.000000,16.651524,-710.973572
9,3385,257.000000,454.000000,302.000000,8300.000000,16.806867,-550.775391


### Parent Deblender Metrics

In [11]:
bundleName = 'parentDeblenderMetrics'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,numParents,numDeblendFailed,numIncompleteData,numDetectedPeaks,numDeblendedChildren
0,2661,146.000000,117.000000,105.000000,6484.000000,0.000000,3466.000000,11585.000000,11585.000000
1,2662,726.000000,450.000000,751.000000,19993.000000,0.000000,5611.000000,42544.000000,42386.000000
2,2703,nan,6.000000,12.000000,443.000000,0.000000,275.000000,552.000000,552.000000
3,2704,nan,nan,23.000000,461.000000,0.000000,0.000000,542.000000,542.000000
4,2876,562.000000,662.000000,557.000000,29049.000000,0.000000,5230.000000,37814.000000,37814.000000
5,2877,2424.000000,2384.000000,2493.000000,83447.000000,0.000000,10630.000000,119631.000000,119591.000000
6,3345,272.000000,276.000000,330.000000,22360.000000,0.000000,5986.000000,38909.000000,38909.000000
7,3346,983.000000,1104.000000,1063.000000,47064.000000,0.000000,8162.000000,90262.000000,90037.000000
8,3384,9506.000000,11103.000000,8263.000000,135314.000000,0.000000,13791.000000,235940.000000,235940.000000
9,3385,257.000000,454.000000,302.000000,10559.000000,0.000000,3537.000000,14712.000000,14712.000000


### PSF CModel Scatter Metrics

In [12]:
bundleName = 'psfCModelScatter'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,g_psf_cModel_diff_median mmag,g_psf_cModel_diff_sigmaMad mmag,g_psf_cModel_diff_mean mmag,r_psf_cModel_diff_median mmag,r_psf_cModel_diff_sigmaMad mmag,r_psf_cModel_diff_mean mmag,i_psf_cModel_diff_median mmag,i_psf_cModel_diff_sigmaMad mmag,i_psf_cModel_diff_mean mmag
0,2661,146.000000,117.000000,105.000000,-1.219545,1.748568,-0.317365,-0.560192,1.557474,0.182773,-0.685470,1.933064,0.348930
1,2662,726.000000,450.000000,751.000000,-0.962125,1.584575,0.358445,-0.413336,1.801341,0.727715,-0.520379,1.689046,0.305720
2,2703,nan,6.000000,12.000000,nan,nan,nan,-0.463216,0.974793,1.200049,8.335912,4.941591,7.107373
3,2704,nan,nan,23.000000,nan,nan,nan,-0.050128,0.943248,1.377076,nan,nan,nan
4,2876,562.000000,662.000000,557.000000,-1.294482,1.759200,-0.475194,-0.815809,1.929917,-0.036058,-1.162528,2.154289,-0.488437
5,2877,2424.000000,2384.000000,2493.000000,-1.817693,1.494304,-1.391281,-0.834848,1.608448,-0.100994,-1.140532,1.683213,-0.556452
6,3345,272.000000,276.000000,330.000000,-0.839224,1.418748,-0.176912,-0.219645,1.579762,0.702652,-0.360833,1.663524,0.265391
7,3346,983.000000,1104.000000,1063.000000,-0.667016,1.308372,0.040122,-0.149433,1.539313,0.722785,-0.385710,1.571954,0.340908
8,3384,9506.000000,11103.000000,8263.000000,-0.849118,1.311614,0.149783,-0.347535,1.637840,0.935007,-0.461571,1.686705,0.390863
9,3385,257.000000,454.000000,302.000000,-1.073353,1.407184,0.712891,-0.400286,1.832108,1.436792,-0.187455,1.658989,0.569315


### Shape Size Fractional Diff Metrics

In [13]:
bundleName = 'shapeSizeFractionalDiff'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct
0,2661,146.000000,117.000000,105.000000,-0.001345,0.003179,35.000000,0.001401,0.003716,783.000000,-0.001794,0.001031,16.000000,-0.000297,0.002903,535.000000,0.000146,0.001664,45.000000,0.000371,0.002960,930.000000
1,2662,726.000000,450.000000,751.000000,-0.000086,0.002864,337.000000,0.001297,0.003327,3715.000000,0.000060,0.001662,479.000000,0.000203,0.002630,4097.000000,0.000152,0.001608,314.000000,0.000416,0.002552,3230.000000
2,2703,nan,6.000000,12.000000,nan,nan,0.000000,nan,nan,0.000000,nan,nan,0.000000,-0.002230,0.002207,4.000000,nan,nan,0.000000,0.007830,0.006350,3.000000
3,2704,nan,nan,23.000000,nan,nan,0.000000,nan,nan,0.000000,0.005080,0.000000,1.000000,-0.002348,0.002266,4.000000,nan,nan,0.000000,nan,nan,0.000000
4,2876,562.000000,662.000000,557.000000,0.001362,0.002760,23.000000,0.001294,0.002833,177.000000,-0.000701,0.001563,30.000000,0.000059,0.002256,231.000000,0.000693,0.002677,43.000000,0.000693,0.002840,313.000000
5,2877,2424.000000,2384.000000,2493.000000,0.001374,0.002609,132.000000,0.001078,0.003313,634.000000,0.000365,0.001690,178.000000,0.000250,0.002281,863.000000,-0.000004,0.001873,181.000000,0.000427,0.002292,955.000000
6,3345,272.000000,276.000000,330.000000,0.001104,0.001802,218.000000,0.001167,0.002667,2512.000000,0.000694,0.001029,350.000000,0.000470,0.001796,2949.000000,0.000791,0.001091,236.000000,0.000637,0.001981,2777.000000
7,3346,983.000000,1104.000000,1063.000000,-0.000018,0.001801,793.000000,0.000774,0.002528,6374.000000,0.000299,0.001254,990.000000,0.000344,0.001925,7484.000000,-0.000447,0.001661,976.000000,0.000329,0.002312,7984.000000
8,3384,9506.000000,11103.000000,8263.000000,0.000535,0.001884,2082.000000,0.000794,0.002512,9161.000000,0.000292,0.001224,2222.000000,0.000493,0.001986,10849.000000,0.000487,0.001398,2815.000000,0.000732,0.002283,13103.000000
9,3385,257.000000,454.000000,302.000000,0.000723,0.002142,31.000000,0.000839,0.002603,367.000000,0.000054,0.001657,38.000000,0.000599,0.002578,563.000000,0.000424,0.000968,51.000000,0.000424,0.002642,695.000000


### Skipped Deblender Metrics

In [14]:
bundleName = 'skippedDeblenderMetrics'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,numSkippedBlends,numBlendParentTooBig,numBlendTooManyPeaks,numBlendTooManyMasked,numSkippedPeaks
0,2661,146.000000,117.000000,105.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2662,726.000000,450.000000,751.000000,2.000000,2.000000,0.000000,0.000000,158.000000
2,2703,nan,6.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2704,nan,nan,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2876,562.000000,662.000000,557.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2877,2424.000000,2384.000000,2493.000000,1.000000,1.000000,0.000000,0.000000,40.000000
6,3345,272.000000,276.000000,330.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,3346,983.000000,1104.000000,1063.000000,3.000000,3.000000,0.000000,0.000000,225.000000
8,3384,9506.000000,11103.000000,8263.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,3385,257.000000,454.000000,302.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Sky Flux Statistic Metrics

In [15]:
bundleName = 'skyFluxStatisticMetric'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,g_medianSky nJy,g_meanSky nJy,g_stdevSky nJy,g_sigmaMADSky nJy,r_medianSky nJy,r_meanSky nJy,r_stdevSky nJy,r_sigmaMADSky nJy,i_medianSky nJy,i_meanSky nJy,i_stdevSky nJy,i_sigmaMADSky nJy
0,2661,146.000000,117.000000,105.000000,9.272572,0.571761,274.445195,184.849933,-14.174829,-6.187000,255.233679,231.362186,-8.449600,17.521631,452.303245,399.738851
1,2662,726.000000,450.000000,751.000000,-4.540783,-0.272711,272.762923,175.518540,1.418278,5.045255,276.201180,189.776241,17.239392,32.432973,456.289132,348.018486
2,2703,nan,6.000000,12.000000,nan,nan,nan,nan,51.848364,104.726647,513.188666,437.194710,10.170679,54.117499,694.514303,482.888378
3,2704,nan,nan,23.000000,nan,nan,nan,nan,-12.705867,34.480325,521.827996,464.114930,nan,nan,nan,nan
4,2876,562.000000,662.000000,557.000000,-25.827921,-25.709258,347.950899,268.351625,-16.884834,-13.976267,361.922825,286.761629,-8.159892,3.924403,425.715814,354.942879
5,2877,2424.000000,2384.000000,2493.000000,-16.805720,-17.756052,278.287160,210.993373,-8.642437,-5.254997,288.374343,226.894587,11.282567,12.264836,345.696247,272.869806
6,3345,272.000000,276.000000,330.000000,-21.035456,-11.364123,230.037490,186.115365,-0.237351,9.995621,272.122999,205.790089,0.488590,-3.621001,341.870530,277.231349
7,3346,983.000000,1104.000000,1063.000000,-6.751929,-2.497408,225.807648,169.287386,-1.866313,4.867158,246.816077,183.915931,2.909271,7.698191,315.467430,238.555886
8,3384,9506.000000,11103.000000,8263.000000,-5.480388,-7.460568,253.347178,166.381950,-11.820947,-10.098930,302.338976,198.169832,7.817764,14.519527,335.116727,206.734367
9,3385,257.000000,454.000000,302.000000,16.064638,1.450232,363.798047,317.284283,-20.658595,9.820325,459.586057,377.105831,-9.248241,-10.605296,482.939203,378.230183


### wPerp Cmodel Metrics

In [16]:
bundleName = 'wPerpCModel'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,wPerp_cModelFlux_sigmaMAD mmag,wPerp_cModelFlux_median mmag
0,2661,146.000000,117.000000,105.000000,12.785274,0.014617
1,2662,726.000000,450.000000,751.000000,14.742439,-1.218877
2,2703,nan,6.000000,12.000000,nan,nan
3,2704,nan,nan,23.000000,nan,nan
4,2876,562.000000,662.000000,557.000000,16.182507,-2.460445
5,2877,2424.000000,2384.000000,2493.000000,12.722163,-0.998216
6,3345,272.000000,276.000000,330.000000,11.919571,-0.336161
7,3346,983.000000,1104.000000,1063.000000,12.200067,-1.078095
8,3384,9506.000000,11103.000000,8263.000000,11.466537,-1.067559
9,3385,257.000000,454.000000,302.000000,18.415226,-0.879265


### wPerp PSFP Metrics

In [17]:
bundleName = 'wPerpPSFP'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,wPerp_psfFlux_sigmaMAD mmag,wPerp_psfFlux_median mmag
0,2661,146.000000,117.000000,105.000000,12.610188,0.240486
1,2662,726.000000,450.000000,751.000000,14.751301,-1.403759
2,2703,nan,6.000000,12.000000,nan,nan
3,2704,nan,nan,23.000000,nan,nan
4,2876,562.000000,662.000000,557.000000,17.281268,-2.161590
5,2877,2424.000000,2384.000000,2493.000000,12.495775,-0.563631
6,3345,272.000000,276.000000,330.000000,11.956128,-0.449725
7,3346,983.000000,1104.000000,1063.000000,12.323952,-0.994844
8,3384,9506.000000,11103.000000,8263.000000,11.494096,-1.059643
9,3385,257.000000,454.000000,302.000000,18.847041,-0.369813


### xPerpCModel Metrics

In [18]:
bundleName = 'xPerpCModel'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,xPerp_cModelFlux_sigmaMAD mmag,xPerp_cModelFlux_median mmag
0,2661,146.000000,117.000000,105.000000,18.428460,4.236078
1,2662,726.000000,450.000000,751.000000,19.115558,-0.180163
2,2703,nan,6.000000,12.000000,nan,nan
3,2704,nan,nan,23.000000,nan,nan
4,2876,562.000000,662.000000,557.000000,12.285779,-1.056423
5,2877,2424.000000,2384.000000,2493.000000,16.528001,0.913770
6,3345,272.000000,276.000000,330.000000,20.636581,-1.196497
7,3346,983.000000,1104.000000,1063.000000,20.311267,-0.728382
8,3384,9506.000000,11103.000000,8263.000000,19.329886,0.377504
9,3385,257.000000,454.000000,302.000000,21.265007,-4.730291


### xPerpPSFP Metrics

In [19]:
bundleName = 'xPerpPSFP'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,xPerp_psfFlux_sigmaMAD mmag,xPerp_psfFlux_median mmag
0,2661,146.000000,117.000000,105.000000,18.005222,4.975893
1,2662,726.000000,450.000000,751.000000,17.933452,0.154265
2,2703,nan,6.000000,12.000000,nan,nan
3,2704,nan,nan,23.000000,nan,nan
4,2876,562.000000,662.000000,557.000000,14.498806,-1.433797
5,2877,2424.000000,2384.000000,2493.000000,17.719898,0.106426
6,3345,272.000000,276.000000,330.000000,20.696450,-1.061882
7,3346,983.000000,1104.000000,1063.000000,20.266560,-0.702573
8,3384,9506.000000,11103.000000,8263.000000,19.221809,0.223788
9,3385,257.000000,454.000000,302.000000,20.515476,-4.447759


## Object Table Extended Metrics

In [20]:
refs = list(registry.queryDatasets('objectTableExtended_metrics'))

### Ap12PSFSky Metrics

In [21]:
bundleName = 'ap12PsfSky'
metricTable, metricMask = metricBundletToDataFrame(bundleName, refs)
displayMaskedTable(tractTable, tractMask, metricTable, metricMask)

,Tract,g,i,r,g_ap12_psf_diff_median mmag,g_ap12_psf_diff_sigmaMad mmag,g_ap12_psf_diff_mean mmag,r_ap12_psf_diff_median mmag,r_ap12_psf_diff_sigmaMad mmag,r_ap12_psf_diff_mean mmag,i_ap12_psf_diff_median mmag,i_ap12_psf_diff_sigmaMad mmag,i_ap12_psf_diff_mean mmag
0,2661,146.000000,117.000000,105.000000,-2.499958,5.626288,-10.070587,-2.598622,5.775990,-12.694267,-3.699441,7.332036,-15.107535
1,2662,726.000000,450.000000,751.000000,-2.416476,5.501507,-7.801497,-2.820403,5.865948,-10.781115,-2.783359,6.885886,-14.194354
2,2703,nan,6.000000,12.000000,nan,nan,nan,0.607158,4.379920,1.775987,-2.026172,1.267285,-2.245531
3,2704,nan,nan,23.000000,nan,nan,nan,3.025515,5.113265,3.430761,nan,nan,nan
4,2876,562.000000,662.000000,557.000000,-0.923078,3.605923,-1.586844,-0.500397,4.054425,-2.057595,-0.854157,4.082398,-2.453589
5,2877,2424.000000,2384.000000,2493.000000,-0.660572,4.276528,-1.061552,-1.083107,3.551634,-2.617272,-1.141631,4.042909,-3.673962
6,3345,272.000000,276.000000,330.000000,-1.178991,4.565035,-4.546323,-1.652228,4.276716,-6.836505,-2.035249,5.098835,-8.910622
7,3346,983.000000,1104.000000,1063.000000,-1.590642,4.161003,-3.538750,-1.725878,4.248605,-5.526495,-2.287039,5.308430,-7.238955
8,3384,9506.000000,11103.000000,8263.000000,-1.186398,3.999158,-2.251845,-1.631396,4.018690,-3.972927,-2.496524,5.240406,-5.842843
9,3385,257.000000,454.000000,302.000000,-0.890114,4.637558,-1.309476,-1.100920,4.293027,-2.234113,-1.706514,4.489825,-3.100053
